# はじめに：このノートブックで学ぶこと

このノートブックでは、RAG（Retrieval-Augmented Generation）の基本的な3つのステップ（チャンキング、検索、生成）の技術的な詳細を、一つ一つ実行しながら学びます。

1.  **ステップA: チャンキング** - 大きなテキストを小さな塊（チャンク）に分割します。
2.  **ステップB: 検索** - 質問と意味的に関連するチャンクを見つけ出します。
3.  **ステップC: 生成** - 検索で見つけた情報を基に、AIが回答を生成します。

### 0. 準備

まず、このチュートリアルで必要となるPythonライブラリをインストールします。

In [1]:
%pip install sentence-transformers openai python-dotenv numpy


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


次に、OpenAI APIを利用するためのAPIキーを設定します。
このノートブックと同じ階層に`.env`という名前のファイルを作成し、その中に`OPENAI_API_KEY='あなたのAPIキー'`と記述してください。

In [2]:
import openai
import os
from dotenv import load_dotenv

# .envファイルを読み込む
load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

if openai.api_key:
    print('OpenAI APIキーが設定されました。')
else:
    print('エラー: .envファイルにOPENAI_API_KEYが見つかりません。')

OpenAI APIキーが設定されました。


学習用のサンプルテキストをインポートします。本ファイルではsample_text_Aのみを使用します。

In [8]:
# 学習用のサンプルテキストを外部ファイルからインポートします
from sample_texts import sample_text_A, sample_text_B, DISPLAY_NAME_A, DISPLAY_NAME_B
use_sample_text = sample_text_A

### 1. ステップA：チャンキング（テキストの分割）

**なぜチャンキングが必要か？**\n
\n
大規模言語モデル（LLM）には、一度に処理できるテキストの長さ（コンテキストウィンドウ）に上限があります。そのため、長い文章をそのまま入力すると、情報が欠落したり、モデルがうまく処理できなかったりします。\n
\n
チャンキングは、元のテキストをモデルが扱いやすいサイズの小さな断片（チャンク）に分割するプロセスです。これにより、関連する情報だけを効率的に検索し、モデルに渡すことができるようになります。

#### 分割方法1：固定文字数

最もシンプルな方法の一つが、テキストを指定した文字数で機械的に分割する方法です。文章の構造を考慮しないため、単語や文の途中で分割されてしまう可能性がありますが、実装は非常に簡単です。

In [ ]:
def chunk_by_size(text, chunk_size):
    """指定された文字数でテキストを分割する関数"""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
chunk_size = 100
chunks_fixed_size = chunk_by_size(use_sample_text, chunk_size)

print(f"--- 固定文字数（{chunk_size}文字）での分割結果 ---")
for i, chunk in enumerate(chunks_fixed_size):
    print(f"チャンク {i+1}: {chunk}")

--- 固定文字数（100文字）での分割結果 ---
チャンク 1: 
遥か未来、人類が宇宙へと進出した時代。
ある小惑星帯に、老夫婦が静かに暮らしていました。
夫はデブリ回収業者として宇宙を飛び回り、妻は小惑星の自宅で水耕栽培をしていました。
ある日、妻がドッキングポ
チャンク 2: ートで宇宙船の洗浄をしていると、
観測史上ないほど巨大な宇宙葡萄の房が、ゆっくりと自転しながら近づいてきました。
その葡萄は、一粒一粒が家ほどもあり、美しい紫色に輝いていました。
「まあ、なんて珍しい
チャンク 3: 葡萄でしょう」。
妻は驚きながらも、マニピュレーターアームを巧みに操り、
その巨大な葡萄の一粒を慎重に回収し、居住ブロックへと運び込みました。
夕方、夫がデブリ回収の仕事を終えて帰還すると、妻はその巨
チャンク 4: 大な葡萄を見せました。
あまりの大きさと美しさに夫も目を見張りました。
「これはきっと、伝説の『創世の葡萄』に違いない。食べれば不老不死になれるという…」。
二人が期待に胸を膨らませ、レーザーカッター
チャンク 5: でその葡萄の厚い皮に切れ込みを入れると、
まばゆい光と共に、中から元気な男の子の赤ちゃんが現れたのです。
宇宙葡萄から生まれたその子を、二人は「葡萄太郎（ぶどうたろう）」と名付けました。
葡萄太郎は、
チャンク 6: 老夫婦の愛情を一身に受け、
小惑星の特殊な環境と栄養豊富な宇宙葡萄のエキスですくすくと育ちました。
彼は生まれながらにして宇宙空間での活動に適応しており、驚異的な身体能力を持っていました。
数年後、た
チャンク 7: くましい青年に成長した葡萄太郎は、
近隣の星系を荒らし回る悪名高い宇宙海賊「ジャークマター」の噂を耳にします。
彼らは貴重な資源を略奪し、平和な植民星を破壊の危機に陥れていました。
「僕が行って、ジャ
チャンク 8: ークマターを懲らしめてきます」。
葡萄太郎の決意は固く、妻は栄養満点の宇宙きび団子を、
夫は最新型の小型宇宙艇を彼に与えました。
旅の途中、葡萄太郎は3体のユニークな仲間と出会います。
最初に訪れたサ
チャンク 9: イバーパンクな機械惑星で、
忠実なAIを搭載した犬型ロボット「イヌ-X」を仲間にしました。
次に立ち寄ったジャングル惑星では、驚異的な知能を持つ猿型サイボーグ「サル-Z」の助けを借り、


#### 分割方法2：改行（`\n`）

箇条書きや詩、コードのように、改行が意味的な区切りとなっているテキストに有効な方法です。

In [10]:
# 空白行を除外するために、strip()で前後の空白を削除した上で、空でないものだけをリストに追加します
chunks_newline = [line for line in use_sample_text.split('\n') if line.strip()] 

print("--- 改行での分割結果 ---")
for i, chunk in enumerate(chunks_newline):
    print(f"チャンク {i+1}: {chunk}")

--- 改行での分割結果 ---
チャンク 1: 遥か未来、人類が宇宙へと進出した時代。
チャンク 2: ある小惑星帯に、老夫婦が静かに暮らしていました。
チャンク 3: 夫はデブリ回収業者として宇宙を飛び回り、妻は小惑星の自宅で水耕栽培をしていました。
チャンク 4: ある日、妻がドッキングポートで宇宙船の洗浄をしていると、
チャンク 5: 観測史上ないほど巨大な宇宙葡萄の房が、ゆっくりと自転しながら近づいてきました。
チャンク 6: その葡萄は、一粒一粒が家ほどもあり、美しい紫色に輝いていました。
チャンク 7: 「まあ、なんて珍しい葡萄でしょう」。
チャンク 8: 妻は驚きながらも、マニピュレーターアームを巧みに操り、
チャンク 9: その巨大な葡萄の一粒を慎重に回収し、居住ブロックへと運び込みました。
チャンク 10: 夕方、夫がデブリ回収の仕事を終えて帰還すると、妻はその巨大な葡萄を見せました。
チャンク 11: あまりの大きさと美しさに夫も目を見張りました。
チャンク 12: 「これはきっと、伝説の『創世の葡萄』に違いない。食べれば不老不死になれるという…」。
チャンク 13: 二人が期待に胸を膨らませ、レーザーカッターでその葡萄の厚い皮に切れ込みを入れると、
チャンク 14: まばゆい光と共に、中から元気な男の子の赤ちゃんが現れたのです。
チャンク 15: 宇宙葡萄から生まれたその子を、二人は「葡萄太郎（ぶどうたろう）」と名付けました。
チャンク 16: 葡萄太郎は、老夫婦の愛情を一身に受け、
チャンク 17: 小惑星の特殊な環境と栄養豊富な宇宙葡萄のエキスですくすくと育ちました。
チャンク 18: 彼は生まれながらにして宇宙空間での活動に適応しており、驚異的な身体能力を持っていました。
チャンク 19: 数年後、たくましい青年に成長した葡萄太郎は、
チャンク 20: 近隣の星系を荒らし回る悪名高い宇宙海賊「ジャークマター」の噂を耳にします。
チャンク 21: 彼らは貴重な資源を略奪し、平和な植民星を破壊の危機に陥れていました。
チャンク 22: 「僕が行って、ジャークマターを懲らしめてきます」。
チャンク 23: 葡萄太郎の決意は固く、妻は栄養満点の宇宙きび団子を、
チャンク 24: 夫は最新型の小型宇宙艇を彼に与えました。
チャンク 25: 旅の途

#### まとめ

このように、テキストの性質（自然な文章か、構造化されたリストかなど）によって、最適な分割方法は異なります。他にも、句読点（。や.）で区切る方法や、より高度な自然言語処理ライブラリ（`spaCy`や`NLTK`）を使って文単位で分割する方法などがあります。

### 2. ステップB：検索（Embedding & Retrieval）

**Embeddingとは何か？**

Embedding（エンベディング）とは、テキスト（単語、文、文章）を、コンピュータが計算できる「ベクトル（数値のリスト）」に変換する技術です。このベクトルの重要な特徴は、**意味が近いテキスト同士は、ベクトル空間上で近い位置に配置される**という点です。

例えば、「犬」と「猫」のベクトルは、「犬」と「机」のベクトルよりも近くなります。これにより、キーワードが完全に一致しなくても、意味的に関連する文章を見つけ出すことが可能になります。

In [11]:
from sentence_transformers import SentenceTransformer

# 事前学習済みのEmbeddingモデルをロードします。
# 'all-MiniLM-L6-v2'は、高速かつ高品質で人気のあるモデルです。
model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/shunsuke_arimura/Desktop/Tech0_desktop/社内Tech0/講義/w9~14_Webアプリ/宿題/RAG_App/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### チャンクと質問をベクトル化する

それでは、先ほど作成したチャンクと、ユーザーからの質問を実際にベクトル化してみましょう。モデルの`encode()`メソッドを使います。
※チャンク化は句読点で分割したものを使用しています。

In [22]:
import numpy as np

# ここでは、句読点で分割したチャンクを使用します
chunks = [p + "。" for p in use_sample_text.split('。') if p.strip()]
question = "葡萄太郎の仲間とが倒した相手は誰ですか？"

# チャンクと質問をベクトル化
chunk_embeddings = model.encode(chunks)
question_embedding = model.encode([question])

print("--- 質問のベクトル（最初の10次元） ---")
print(question_embedding[0, :10])
print("ベクトルの形状:", question_embedding.shape)

print("--- 最初のチャンクのベクトル（最初の10次元） ---")
print(chunk_embeddings[0, :10])
print("ベクトルの形状:", chunk_embeddings.shape)

--- 質問のベクトル（最初の10次元） ---
[-0.02801427  0.08815496  0.04257226  0.01339853 -0.04357269  0.10896344
  0.04952611  0.04370211 -0.06755138 -0.05930566]
ベクトルの形状: (1, 384)
--- 最初のチャンクのベクトル（最初の10次元） ---
[-0.02559412  0.04930006  0.04341166 -0.02697592 -0.02838277  0.055016
  0.03523876  0.02062199 -0.01630171  0.02270142]
ベクトルの形状: (31, 384)


↑このように、各テキストが高次元の数字のリスト（ベクトル）に変換されていることがわかります。

#### コサイン類似度で「意味の近さ」を計算する

テキストをベクトル化できたら、次はその「近さ」を計算します。ベクトル空間における「近さ」を測る指標はいくつかありますが、最も一般的に使われるのが**コサイン類似度**です。

コサイン類似度は、2つのベクトルが指す方向がどれだけ似ているかを示します。値は-1から1の範囲を取り、1に近いほど「似ている」と判断されます。

In [23]:
from sentence_transformers.util import cos_sim

# 質問ベクトルと全チャンクベクトルのコサイン類似度を計算
similarities = cos_sim(question_embedding, chunk_embeddings)

print("--- 各チャンクと質問の類似度スコア ---")
print(similarities)

--- 各チャンクと質問の類似度スコア ---
tensor([[0.5944, 0.6148, 0.5402, 0.5106, 0.5628, 0.5467, 0.5365, 0.6466, 0.5357,
         0.4733, 0.5089, 0.4661, 0.6547, 0.6442, 0.5899, 0.5190, 0.6060, 0.5488,
         0.6352, 0.7734, 0.6154, 0.5272, 0.7344, 0.5943, 0.5203, 0.6067, 0.6315,
         0.5165, 0.7375, 0.5518, 0.5783]])


In [25]:
# スコアとチャンクをペアにして、スコアの高い順に並べ替える
scored_chunks = sorted(zip(chunks, similarities.flatten()), key=lambda x: x[1], reverse=True)

print("--- 関連度の高い順に並べ替えたチャンク ---")
for i, (chunk, score) in enumerate(scored_chunks):
    print(f"【順位 {i+1} / スコア: {score:.4f}】")
    print(chunk.strip())
    print("---")


--- 関連度の高い順に並べ替えたチャンク ---
【順位 1 / スコア: 0.7734】
旅の途中、葡萄太郎は3体のユニークな仲間と出会います。
---
【順位 2 / スコア: 0.7375】
激しい戦闘の末、葡萄太郎は仲間たちとの連携プレイで首領を打ち破り、
ジャークマターを降伏させました。
---
【順位 3 / スコア: 0.7344】
葡萄太郎と3体の仲間たちは、宇宙艇でジャークマターの本拠地である暗黒星雲へと向かいました。
---
【順位 4 / スコア: 0.6547】
宇宙葡萄から生まれたその子を、二人は「葡萄太郎（ぶどうたろう）」と名付けました。
---
【順位 5 / スコア: 0.6466】
夕方、夫がデブリ回収の仕事を終えて帰還すると、妻はその巨大な葡萄を見せました。
---
【順位 6 / スコア: 0.6442】
葡萄太郎は、老夫婦の愛情を一身に受け、
小惑星の特殊な環境と栄養豊富な宇宙葡萄のエキスですくすくと育ちました。
---
【順位 7 / スコア: 0.6352】
葡萄太郎の決意は固く、妻は栄養満点の宇宙きび団子を、
夫は最新型の小型宇宙艇を彼に与えました。
---
【順位 8 / スコア: 0.6315】
首領は巨大な体を持つ恐ろしい異星人でした。
---
【順位 9 / スコア: 0.6154】
最初に訪れたサイバーパンクな機械惑星で、
忠実なAIを搭載した犬型ロボット「イヌ-X」を仲間にしました。
---
【順位 10 / スコア: 0.6148】
ある小惑星帯に、老夫婦が静かに暮らしていました。
---
【順位 11 / スコア: 0.6067】
母船の内部は迷路のように入り組んでいましたが、
イヌ-Xの解析能力、サル-Zのハッキング技術、キジ-Vの偵察能力を駆使して、
葡萄太郎はついに首領の部屋へとたどり着きました。
---
【順位 12 / スコア: 0.6060】
彼らは貴重な資源を略奪し、平和な植民星を破壊の危機に陥れていました。
---
【順位 13 / スコア: 0.5944】
遥か未来、人類が宇宙へと進出した時代。
---
【順位 14 / スコア: 0.5943】
そこは無数のアステロイドが飛び交う危険な宙域でした。
---
【順位 15 / スコア: 0.5899】
彼は生まれながらにして宇宙空間での

↑「葡萄太郎、仲間、敵という言葉から連想される言葉」という部分を含むチャンクのスコアが最も高くなっていることがわかります。

### 3. ステップC：生成（Generation）

**RAGの核心：プロンプトの「拡張」(Augmented)**

いよいよ最終ステップです。ここで行うのは、検索で見つけ出した関連性の高いチャンク（＝コンテキスト）を、元の質問と組み合わせて、LLMへの**プロンプトを「拡張」する**ことです。

これにより、LLMはゼロから答えを「思い出す」のではなく、与えられたコンテキストという「カンニングペーパー」を基に、より正確な回答を生成できるようになります。これが、RAGがハルシネーション（事実に基づかない情報の生成）を抑制できる理由です。

In [26]:
# OpenAIのクライアントを初期化
client = openai.OpenAI()

#### 「RAGあり」プロンプトの組み立て\n
\n
ステップBでスコアが高かった上位3件のチャンクを「コンテキスト」として、最終的なプロンプトを組み立ててみましょう。

In [29]:
# 上位3件のチャンクをコンテキストとして使用\n
top_n = 3
context = "\n\n".join([chunk for chunk, score in scored_chunks[:top_n]])

# プロンプトテンプレート
prompt_template = f"""
以下の参考情報を使って、質問に答えてください。参考情報に答えがない場合は、「分かりません」と答えてください。

--- 
【参考情報】
{context}
--- 
【質問】
{question}
"""

final_prompt = prompt_template.format(context=context, question=question)

print("--- AIに渡す最終プロンプト（RAGあり） ---")
print(final_prompt)

--- AIに渡す最終プロンプト（RAGあり） ---

以下の参考情報を使って、質問に答えてください。参考情報に答えがない場合は、「分かりません」と答えてください。

--- 
【参考情報】

旅の途中、葡萄太郎は3体のユニークな仲間と出会います。


激しい戦闘の末、葡萄太郎は仲間たちとの連携プレイで首領を打ち破り、
ジャークマターを降伏させました。


葡萄太郎と3体の仲間たちは、宇宙艇でジャークマターの本拠地である暗黒星雲へと向かいました。
--- 
【質問】
葡萄太郎の仲間とが倒した相手は誰ですか？



In [30]:
# 「RAGありプロンプト」で回答を生成\n
print("\n--- 「RAGあり」での回答生成 ---")
response_rag = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": "You are a helpful assistant." },
        {"role": "user", "content": final_prompt}
    ]
)
print(response_rag.choices[0].message.content)


--- 「RAGあり」での回答生成 ---
首領です。仲間と協力して首領を打ち破り、その後ジャークマターを降伏させました。


#### 比較：RAGなしの場合

比較のために、コンテキストを与えずに、ユーザーの質問だけをLLMに渡してみましょう。

In [31]:
# 「RAGなしプロンプト」（質問のみ）
prompt_no_rag = question

print("--- AIに渡すプロンプト（RAGなし） ---")
print(prompt_no_rag)

print("\n--- 「RAGなし」での回答生成 ---")
response_no_rag = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": "You are a helpful assistant." },
        {"role": "user", "content": prompt_no_rag}
    ]
)
print(response_no_rag.choices[0].message.content)

--- AIに渡すプロンプト（RAGなし） ---
葡萄太郎の仲間とが倒した相手は誰ですか？

--- 「RAGなし」での回答生成 ---
おそらく「桃太郎」のことを指していると思います。もし別の作品のことなら教えてください。

桃太郎と仲間（犬・猿・雉）が倒した相手は鬼（おに）です。鬼ヶ島の鬼を討伐します。


#### 考察：RAGあり/なしの回答を比較してみましょう

2つの回答を比べてみると、以下のことが分かります。

*   **RAGありの場合:** 与えられた「参考情報」に忠実に、葡萄太郎の内容について回答しています。
*   **RAGなしの場合:** モデルが元々持っている知識から回答を生成しようとします。葡萄太郎はこのために作ったデタラメな物語であるため、正しく回答することはできません。

このように、RAGは、**外部の信頼できる情報源を基に回答を生成する**ことで、LLMの回答の正確性と信頼性を向上させる強力な技術です。

### おまけ

PDFファイルをベクトル化し、supabaseにベクトルデータとして保管できるようにしてみましょう